# Playing with Boxes

In this notebook, we will perform the following tasks in **pytorch as well as tensorflow**:
1. Create a tensor representation of a box
2. Represent specific boxes using your tensor representation
3. Create a function which calculates the intersection of two boxes
3. Create a differentiable function to compute "soft" volumes of boxes
4. Train one box to contain another


This notebook is intended to be self-contained, but you may find it beneficial to consult Section 3.1 to 3.4 in [Representing Joint Hierarchies with Box Embeddings](https://openreview.net/pdf?id=J246NSqR_l).

## 1. Create a box parameterization

A "box" is a product of intervals in $\mathbb R^n$, i.e.

$$X = \prod_{i=1}^d [x_{m,i}, x_{M,i}], \quad \text{where} \quad x_{M,i} \ge x_{m,i} \quad \text{ for all } \quad i.$$

However you would like, create a way of storing parameters which represent a box. Crucially, your parameterization should conform to the requirement above, namely any setting of your parameterization should represent a box that has positive side-lengths in each dimension, and you should be able to represent any box in $\mathbb R^n$. Furthermore, you should be able to return the min and max coordinates (i.e. the $x_{m,i}, x_{M, i}$ above) for your box. 

**Hint**: It might be benificial to create a "wrapper" class that holds the tensor(s) corresponding to the parameters of the box.

In [23]:
# PyTorch version
import torch

# Wrapper class
class Box_torch(object):
    def __init__(self, x_m, x_M):
        """
        x_m: min coordinate
        x_M: max coordinate
        """
        self.x_m = x_m
        self.x_M = x_M
    def create_box(self):
        """
        create a torch tensor box
        """
        box = torch.tensor([self.x_m, self.x_M], dtype=torch.float32)
        return box

box_method = Box_torch([1,2,3,4], [7,8,9,10])
box_method.create_box()

tensor([[ 1.,  2.,  3.,  4.],
        [ 7.,  8.,  9., 10.]])

In [24]:
# TensorFlow version
import tensorflow as tf

# Wrapper class
class Box_tf(object):
    def __init__(self, x_m, x_M):
        """
        x_m: min coordinate
        x_M: max coordinate
        """
        self.x_m = x_m
        self.x_M = x_M
    def create_box(self):
        """
        create a Tf tensor box
        """
        box = tf.Variable([self.x_m, self.x_M], dtype=tf.float32)
        return box

box_method = Box_tf([1,2,3,4], [7,8,9,10])
box_method.create_box()

<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[ 1.,  2.,  3.,  4.],
       [ 7.,  8.,  9., 10.]], dtype=float32)>

## 2. Represent Specific Boxes
Using your chosen parameterization, represent a box `x` in $\mathbb R^{100}$ which has side-lengths $2$ centered at $(-1,\ldots, -1)$. 
Create another box `y` with min-coordinate $(1, \frac 1 2, \frac 1 3, \ldots, \frac 1 {100})$, and max-coordinate $(3 - \frac 1 {100}, 3 - \frac 1 {99}, \ldots, 2)$.


In [25]:
# PyTorch version
# Represent a specific box x
import numpy as np
dim = 100
center = np.array([-1.0]*dim)
x_length = np.array([2.0]*dim)
box_method = Box_torch(center - (x_length/2.0), center + (x_length/2.0))
x_torch = box_method.create_box()

# Represent a specific box y
import numpy as np
y_m = 1/np.array(list(range(1, 101)))
y_M = 3 - 1/np.array(list(range(100, 0, -1)))
box_method = Box_torch(y_m, y_M)
y_torch = box_method.create_box()

In [26]:
# TensorFlow version
# Represent a specific box x
import numpy as np
dim = 100
center = np.array([-1.0]*dim)
x_length = np.array([2.0]*dim)
box_method = Box_tf(center - (x_length/2.0), center + (x_length/2.0))
x_tf = box_method.create_box()

# Represent a specific box y
y_m = 1/np.array(list(range(1, 101)))
y_M = 3 - 1/np.array(list(range(100, 0, -1)))
box_method = Box_tf(y_m, y_M)
y_tf = box_method.create_box()

In [27]:
print('x torch: ', x_torch)
print('y torch: ', y_torch)
print('x tf: ', x_tf)
print('y tf: ', y_tf)

x torch:  tensor([[-2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
         -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
         -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
         -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
         -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
         -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
         -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
         -2., -2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.

## 3. Create an Intersection Function

The box intersection operation takes two boxes as input and returns their intersection. In one-dimension, this is as follows:

$$\text{Int}([x_m, x_M], [y_m, y_M]) = (\max(x_m, y_m), \min(x_M, y_M)).$$

Generalize this to $n$-dimensions, and write an intersection function which operates on your box parameterization.

Take the intersection of the two boxes `x` and `y` you created in step 2, and display the min and max coordinates of the intersection.

In [28]:
def intersection_torch (x,y):
    """
    this function takes two boxes x,y as input and returns their intersection
    """
    max_coor = torch.max(x[0], y[0])
    min_coor = torch.min(x[1], y[1])
    return (max_coor, min_coor)

def intersection_tf(x,y):
    """
    this function takes two boxes x,y as input and returns their intersection
    """
    max_coor = tf.math.maximum(x[0], y[0])
    min_coor = tf.math.minimum(x[1], y[1])
    return (max_coor, min_coor)

In [29]:
intersection_torch (x_torch, y_torch)

(tensor([1.0000, 0.5000, 0.3333, 0.2500, 0.2000, 0.1667, 0.1429, 0.1250, 0.1111,
         0.1000, 0.0909, 0.0833, 0.0769, 0.0714, 0.0667, 0.0625, 0.0588, 0.0556,
         0.0526, 0.0500, 0.0476, 0.0455, 0.0435, 0.0417, 0.0400, 0.0385, 0.0370,
         0.0357, 0.0345, 0.0333, 0.0323, 0.0312, 0.0303, 0.0294, 0.0286, 0.0278,
         0.0270, 0.0263, 0.0256, 0.0250, 0.0244, 0.0238, 0.0233, 0.0227, 0.0222,
         0.0217, 0.0213, 0.0208, 0.0204, 0.0200, 0.0196, 0.0192, 0.0189, 0.0185,
         0.0182, 0.0179, 0.0175, 0.0172, 0.0169, 0.0167, 0.0164, 0.0161, 0.0159,
         0.0156, 0.0154, 0.0152, 0.0149, 0.0147, 0.0145, 0.0143, 0.0141, 0.0139,
         0.0137, 0.0135, 0.0133, 0.0132, 0.0130, 0.0128, 0.0127, 0.0125, 0.0123,
         0.0122, 0.0120, 0.0119, 0.0118, 0.0116, 0.0115, 0.0114, 0.0112, 0.0111,
         0.0110, 0.0109, 0.0108, 0.0106, 0.0105, 0.0104, 0.0103, 0.0102, 0.0101,
         0.0100]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [30]:
intersection_tf (x_tf, y_tf)

(<tf.Tensor: shape=(100,), dtype=float32, numpy=
 array([1.        , 0.5       , 0.33333334, 0.25      , 0.2       ,
        0.16666667, 0.14285715, 0.125     , 0.11111111, 0.1       ,
        0.09090909, 0.08333334, 0.07692308, 0.07142857, 0.06666667,
        0.0625    , 0.05882353, 0.05555556, 0.05263158, 0.05      ,
        0.04761905, 0.04545455, 0.04347826, 0.04166667, 0.04      ,
        0.03846154, 0.03703704, 0.03571429, 0.03448276, 0.03333334,
        0.03225806, 0.03125   , 0.03030303, 0.02941176, 0.02857143,
        0.02777778, 0.02702703, 0.02631579, 0.02564103, 0.025     ,
        0.02439024, 0.02380952, 0.02325581, 0.02272727, 0.02222222,
        0.02173913, 0.0212766 , 0.02083333, 0.02040816, 0.02      ,
        0.01960784, 0.01923077, 0.01886792, 0.01851852, 0.01818182,
        0.01785714, 0.01754386, 0.01724138, 0.01694915, 0.01666667,
        0.01639344, 0.01612903, 0.01587302, 0.015625  , 0.01538462,
        0.01515152, 0.01492537, 0.01470588, 0.01449275, 0.01428571,

## 4. Create a "Soft" Volume Function
The "softplus volume" of a box is defined as

$$\text{SoftVol}[X] = \prod_{i=1}^d \log(1 + \exp(x_{M,i} - x_{m,i})).$$

Create this volume function, and attempt to use it to determine which of the boxes `x` and `y` from part (2) are larger. What issues do you encounter? Can you mitigate these issues?

In [31]:
# Soft Volume Function - Pytorch Version
import torch
def softplus_vol_torch(m, M):
    """
    given min and max coordinates of a box, compute soft volumn
    """
    vol = torch.prod(torch.log(1 + torch.exp(M-m)))
    return vol

#---------------------

# Soft Volume Function - TensorFlow Version
import tensorflow as tf
def softplus_vol_tf(m, M):
    """
    given min and max coordinates of a box, compute soft volumn
    """
    vol = tf.math.reduce_prod(tf.math.log(1+tf.math.exp(M-m)))
    return vol

In [32]:
print(softplus_vol_torch(x_torch[0], x_torch[1]))
print(softplus_vol_tf(x_tf[0], x_tf[1]))
print('-----------')
print(softplus_vol_torch(y_torch[0], y_torch[1]))
print(softplus_vol_tf(y_tf[0], y_tf[1]))

tensor(5.9605e+32)
tf.Tensor(5.9604763e+32, shape=(), dtype=float32)
-----------
tensor(inf)
tf.Tensor(inf, shape=(), dtype=float32)


I noticed that if I use `float32`, the volumn of y will be `Inf` (Too big to display). If I use `float64`, I can see the exact answer. However, `float64` will be very expensive for calculation later on. Inspired by the methods to tackle gradient overflow in training DNN models, we can try:  
- **Scaling**: We can try to normalize the vector  $(x_{M,i} - x_{m,i})$ (i.e. vector with a predefined norm such as 1.0)
- **Clipping**: We can force the volumn to be a specific maximum value if it exceeds an expected value.

## 5. Training Boxes
We say box $X$ contains box $Y$ if, for each dimension $i$, we have
$$x_{m,i} < y_{m,i} < y_{M,i} < x_{M,i}.$$

Freezing the coordinates of box `x` from earlier, use gradient-descent to train box `y` such that it is contained in `x`.

**Hint:** Visualize typical boxes in 2-dimensions, and note that if a box $Y$ were contained in $X$ then $\text{Vol}(\text{Int}(X, Y)) = \text{Vol}(Y)$.

**Hint:** You can use/invent any loss function as long as it has a local minima that corresponds to satisfying the inequality given above.

**Comment** The way I set up the loss is probably wrong, so I didn't achieve what was asked for this question

In [34]:
# Training boxes
import torch.optim as optim
import torch.nn as nn
dtype = torch.float32
device = torch.device('cpu')
print('using device:', device)
learning_rate = 1e-2

def train(x, y, optimizer, epochs=1):
    if softplus_vol_torch(intersection_torch(x,y)[0], intersection_torch(x,y)[1]) == softplus_vol_torch(y[0], y[1]):
        print ('success: y is already contained in x!')
    else:
        for e in range(epochs):
            x_m = x[0].to(device=device, dtype=dtype).requires_grad_()
            x_M = x[1].to(device=device, dtype=dtype).requires_grad_()
            y_m = y[0].to(device=device, dtype=dtype).requires_grad_()
            y_M = y[1].to(device=device, dtype=dtype).requires_grad_()
            loss = torch.sum(torch.square(x_M-y_M+y_m-x_m)) # Least square loss
            optimizer.zero_grad()
            loss.backward()    
            optimizer.step()
            print('Iteration %d, loss = %.4f' % (e, loss.item()))

optimizer =  torch.optim.SGD([y_torch[0], y_torch[1]], lr=learning_rate)
train(x_torch, y_torch, optimizer, epochs=10)

using device: cpu
Iteration 0, loss = 82.7259
Iteration 1, loss = 82.7259
Iteration 2, loss = 82.7259
Iteration 3, loss = 82.7259
Iteration 4, loss = 82.7259
Iteration 5, loss = 82.7259
Iteration 6, loss = 82.7259
Iteration 7, loss = 82.7259
Iteration 8, loss = 82.7259
Iteration 9, loss = 82.7259


In [35]:
# Training boxes
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
dtype = torch.float32
device = '/cpu:0'
print('Using device: ', device)

optimizer = keras.optimizers.SGD(learning_rate=1e-2)

def train(x, y, optimizer, epochs=1):
    if softplus_vol_tf(intersection_tf(x,y)[0], intersection_tf(x,y)[1]) == softplus_vol_tf(y[0], y[1]):
        print ('success: y is already contained in x!')
    else:
        for e in range(epochs):
             with tf.GradientTape() as tape:
                x_m = x[0]
                x_M = x[1]
                y_m = y[0]
                y_M = y[1]
                loss = tf.math.reduce_sum(tf.square(x_M-y_M+y_m-x_m)) # Least square loss
                grads = tape.gradient(loss, [y_m, y_M])
                
                print('Iteration %d, loss = %.4f' % (e, loss))

train(x_tf, y_tf, optimizer, epochs=10)

Using device:  /cpu:0
Iteration 0, loss = 82.7259
Iteration 1, loss = 82.7259
Iteration 2, loss = 82.7259
Iteration 3, loss = 82.7259
Iteration 4, loss = 82.7259
Iteration 5, loss = 82.7259
Iteration 6, loss = 82.7259
Iteration 7, loss = 82.7259
Iteration 8, loss = 82.7259
Iteration 9, loss = 82.7259
